In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import csv
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
import os
from PIL import Image
import torchvision.models as models
%matplotlib inline
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [0]:
class MuraBodyDataset(torch.utils.data.Dataset):
    
    def __init__(self,text_file,root_dir, transform ): #Normalize to pretrained VGGNet
        """
        Args:
            text_file(string): path to text file
            root_dir(string): directory with all train images
        """
        #File with the Path
        self.name_frame = pd.read_csv(text_file,sep=",",usecols=[0],dtype = 'str')
        #File with labels
        self.label_frame = pd.read_csv(text_file,sep=",", usecols=[1])
        self.root_dir = root_dir
        self.transform = transform
                                       
    def __len__(self):
        return len(self.name_frame)

    def __getitem__(self, idx):
        #Pull image
        img_name = os.path.join(self.root_dir, self.name_frame.iloc[idx, 0])
        if not os.path.exists(img_name):
          print(img_name)
        else:        
          #Apply adapative threshold to highlight key features in the image   
          image = cv2.imread(img_name,0)
          image = cv2.adaptiveThreshold(image,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,11,2)  
          image = Image.fromarray(image)
          image = image.convert('L')

          #Apply tensor transformations to images
          image = self.transform(image) 

          # I included labels in the train_image_path file in excel using a find("positive")
          labels = self.label_frame.iloc[idx,0] 

          #Formatting for labels
          labels = np.array(labels)
          labels = np.reshape(labels, (1,1))
          labels= torch.from_numpy(labels.astype('int'))

          sample = {'image': image, 'labels': labels}
        
        return sample

In [0]:
MuraTrainSet = MuraBodyDataset(text_file ='/content/gdrive/My Drive/MURA-v1.1/Forearm_Mura.csv',
                               root_dir = '/content/gdrive/My Drive',
                               transform = transforms.Compose([transforms.Resize((224,224)),
                                                          transforms.ToTensor(),
                                                          transforms.Normalize(
                                                              mean=[0.456],
                                                              std=[0.224])]))

MuraTrainLoader = torch.utils.data.DataLoader(MuraTrainSet,batch_size=32,shuffle=True, num_workers=0)

MuraTestSet = MuraBodyDataset(text_file ='/content/gdrive/My Drive/MURA-v1.1/Forearm_Mura_test.csv',
                              root_dir = '/content/gdrive/My Drive',
                              transform = transforms.Compose([transforms.Resize((224,224)),
                                                          transforms.ToTensor(),
                                                          transforms.Normalize(
                                                              mean=[0.456],
                                                              std= [0.225])]))

MuraTestLoader = torch.utils.data.DataLoader(MuraTestSet,batch_size=32,shuffle=False, num_workers=0)

In [5]:
dense_net169 = models.densenet169(pretrained=True)

for param in dense_net169.features.parameters():
    param.requires_grad = False
    
dense_net169.features._modules['conv0']  = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
#Adjust to grayscale = 1 channel. Requires gradient is True by default
dense_net169.classifier = nn.Linear(in_features=1664, out_features=1, bias=True)
dense_net169.to(device)


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

In [0]:
def train(epoch, max_batch, loss_array, accuracy_array):
    dense_net169.train()
    train_loss = 0
    TN = 0
    TP = 0
    FP = 0
    FN = 0
    for i, sample_batched in enumerate(MuraTrainLoader,1):
        if i <max_batch+1:
        
            inputs = sample_batched['image']
            labels = sample_batched['labels']
            inputs, labels = inputs.to(device), labels.to(device)
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = dense_net169(inputs)
            labels = labels.view(len(sample_batched['labels']),-1)
            loss = criterion(outputs.float(), labels.float())
            loss.backward()
            optimizer.step()
            train_loss+= loss.item()

            pred = (torch.sigmoid(outputs).data > 0.5).int()
            labels = labels.int()
            TP += ((pred == 1) & (labels == 1) ).float().sum() #True Positive Count
            TN += ((pred == 0) & (labels == 0) ).float().sum() #True Negative Count
            FP += ((pred == 1) & (labels == 0) ).float().sum() #False Positive Count
            FN += ((pred == 0) & (labels == 1) ).float().sum() #False Negative Count
        else:
            print('Batches completed: {}'.format(max_batch))
            max_batch=max_batch+max_batch
            
    
    accuracy = (TP + TN)/(TP + TN + FP + FN)
    precision = TP/(TP + FP)
    recall = TP/(TP + FN)
    f1_score = 2 * (precision*recall)/(precision+recall)
    p_yes = ((TP + FP)/(TP + TN + FP + FN))*((TP + FN)/(TP + TN + FP + FN))
    p_no = ((TN + FN)/(TP + TN + FP + FN))*((FP + TN)/(TP + TN + FP + FN))
    p_e = p_yes+p_no
    kappa = (accuracy - p_e)/(1-p_e)
    accuracy_array.append([TP, TN, FP, FN, accuracy, precision, recall, f1_score])
    train_loss = float(train_loss)/float(i)
    loss_array.append(train_loss)
    # print statistics        
    print('Train Epoch:{}  Accuracy: {:.2f}   Average Loss: {:.2f}  Precision: {:.2f}   F1 Score: {:.2f} Kappa: {:.2f}\n'.
          format(epoch, accuracy, train_loss, precision, f1_score, kappa))


In [0]:
learning = 0.0001  # Learning Rate 
w = 1e-1
dense_train_loss = []
dense_train_accuracy =[]
criterion = nn.BCEWithLogitsLoss() 
optimizer = optim.Adam(dense_net169.parameters(), lr=learning, weight_decay = w)

for epoch in range(1,21): 
    train(epoch,1000, dense_train_loss,dense_train_accuracy)

Train Epoch:1  Accuracy: 0.60   Average Loss: 0.68  Precision: 0.34   F1 Score: 0.17

Train Epoch:2  Accuracy: 0.64   Average Loss: 0.65  Precision: 0.58   F1 Score: 0.04

Train Epoch:3  Accuracy: 0.65   Average Loss: 0.64  Precision: 0.64   F1 Score: 0.14

Train Epoch:4  Accuracy: 0.65   Average Loss: 0.63  Precision: 0.71   F1 Score: 0.13

Train Epoch:5  Accuracy: 0.66   Average Loss: 0.62  Precision: 0.67   F1 Score: 0.22

Train Epoch:6  Accuracy: 0.67   Average Loss: 0.61  Precision: 0.71   F1 Score: 0.25

Train Epoch:7  Accuracy: 0.68   Average Loss: 0.60  Precision: 0.70   F1 Score: 0.33

Train Epoch:8  Accuracy: 0.69   Average Loss: 0.60  Precision: 0.72   F1 Score: 0.35

Train Epoch:9  Accuracy: 0.70   Average Loss: 0.60  Precision: 0.73   F1 Score: 0.38

Train Epoch:10  Accuracy: 0.70   Average Loss: 0.59  Precision: 0.74   F1 Score: 0.38

Train Epoch:11  Accuracy: 0.70   Average Loss: 0.59  Precision: 0.73   F1 Score: 0.41

Train Epoch:12  Accuracy: 0.70   Average Loss: 0.58 

In [0]:
for epoch in range(1,31): 
    train(epoch,1000, dense_train_loss,dense_train_accuracy)

Train Epoch:1  Accuracy: 0.73   Average Loss: 0.54  Precision: 0.75   F1 Score: 0.49 Kappa: 0.33

Train Epoch:2  Accuracy: 0.74   Average Loss: 0.54  Precision: 0.77   F1 Score: 0.55 Kappa: 0.39

Train Epoch:3  Accuracy: 0.74   Average Loss: 0.54  Precision: 0.77   F1 Score: 0.52 Kappa: 0.37

Train Epoch:4  Accuracy: 0.75   Average Loss: 0.53  Precision: 0.80   F1 Score: 0.54 Kappa: 0.39

Train Epoch:5  Accuracy: 0.75   Average Loss: 0.52  Precision: 0.77   F1 Score: 0.57 Kappa: 0.41

Train Epoch:6  Accuracy: 0.75   Average Loss: 0.53  Precision: 0.78   F1 Score: 0.55 Kappa: 0.39

Train Epoch:7  Accuracy: 0.75   Average Loss: 0.52  Precision: 0.78   F1 Score: 0.56 Kappa: 0.40

Train Epoch:8  Accuracy: 0.76   Average Loss: 0.51  Precision: 0.80   F1 Score: 0.58 Kappa: 0.44

Train Epoch:9  Accuracy: 0.75   Average Loss: 0.51  Precision: 0.79   F1 Score: 0.55 Kappa: 0.40

Train Epoch:10  Accuracy: 0.77   Average Loss: 0.51  Precision: 0.81   F1 Score: 0.61 Kappa: 0.46

Train Epoch:11  Acc

In [0]:
PATH = '/content/gdrive/My Drive/MURA-v1.1/forearm_1.pth'
torch.save(dense_net169.state_dict(), PATH)

In [0]:
def test(epoch, loss_array, accuracy_array, pred_array):
    #Have our model in evaluation mode
    dense_net169.eval()
    #Set losses and Correct labels to zero
    test_loss = 0
    TN = 0
    TP = 0
    FP = 0
    FN = 0
    with torch.no_grad():
        for i, sample_batched in enumerate(MuraTestLoader,1):
            inputs = sample_batched['image']
            labels = sample_batched['labels']
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = dense_net169(inputs)
            labels = labels.view(len(sample_batched['labels']),-1)
            loss = criterion(outputs.float(), labels.float())
            test_loss += loss.item()

            pred = (torch.sigmoid(outputs).data > 0.5).int()
            pred_array.append(pred_array)
            labels = labels.int()
            TP += ((pred == 1) & (labels == 1) ).float().sum() #True Positive Count
            TN += ((pred == 0) & (labels == 0) ).float().sum() #True Negative Count
            FP += ((pred == 1) & (labels == 0) ).float().sum() #False Positive Count
            FN += ((pred == 0) & (labels == 1) ).float().sum() #False Negative Count
            #print('TP: {}\t TN: {}\t FP: {}\t FN: {}\n'.format(TP,TN,FP,FN) )    

        accuracy = (TP + TN)/(TP + TN + FP + FN)
        precision = TP/(TP + FP)
        recall = TP/(TP + FN)
        f1_score = 2 * (precision*recall)/(precision+recall)
        accuracy_array.append([TP, TN, FP, FN, accuracy, precision, recall, f1_score])
        test_loss = float(test_loss)/float(i)
        loss_array.append(test_loss)
        # print statistics        
        print('Test  Epoch:{}  Accuracy: {:.2f}   Average Loss: {:.2f}  Precision: {:.2f}   F1 Score: {:.2f}\n'.
              format(epoch, accuracy, test_loss, precision, f1_score))

In [0]:
dense_test_loss = []
dense_test_accuracy = []
dense_test_pred =[]
for epoch in range(1,2): 
    test(epoch, dense_test_loss,dense_test_accuracy, dense_test_pred )

Test  Epoch:1  Accuracy: 0.66   Average Loss: 0.67  Precision: 0.80   F1 Score: 0.55



**Shoulder Data**

In [0]:
MuraTrainSet = MuraBodyDataset(text_file ='/content/gdrive/My Drive/MURA-v1.1/Elbow_Mura.csv',
                               root_dir = '/content/gdrive/My Drive',
                               transform = transforms.Compose([transforms.Resize((224,224)),
                                                          transforms.ToTensor(),
                                                          transforms.Normalize(
                                                              mean=[0.456],
                                                              std=[0.224])]))

MuraTrainLoader = torch.utils.data.DataLoader(MuraTrainSet,batch_size=32,shuffle=True, num_workers=0)

MuraTestSet = MuraBodyDataset(text_file ='/content/gdrive/My Drive/MURA-v1.1/Elbow_Mura_Test.csv',
                              root_dir = '/content/gdrive/My Drive',
                              transform = transforms.Compose([transforms.Resize((224,224)),
                                                          transforms.ToTensor(),
                                                          transforms.Normalize(
                                                              mean=[0.456],
                                                              std= [0.225])]))

MuraTestLoader = torch.utils.data.DataLoader(MuraTestSet,batch_size=32,shuffle=False, num_workers=0)

In [0]:
learning = 0.0001  # Learning Rate 
w = 1e-1
dense_train_loss = []
dense_train_accuracy =[]
criterion = nn.BCEWithLogitsLoss() 
optimizer = optim.Adam(dense_net169.parameters(), lr=learning, weight_decay = w)

for epoch in range(1,51): 
    train(epoch,1000, dense_train_loss,dense_train_accuracy)

Train Epoch:1  Accuracy: 0.61   Average Loss: 0.66  Precision: 0.61   F1 Score: 0.23 Kappa: 0.09

Train Epoch:2  Accuracy: 0.63   Average Loss: 0.64  Precision: 0.63   F1 Score: 0.34 Kappa: 0.15

Train Epoch:3  Accuracy: 0.64   Average Loss: 0.63  Precision: 0.65   F1 Score: 0.38 Kappa: 0.19

Train Epoch:4  Accuracy: 0.67   Average Loss: 0.62  Precision: 0.70   F1 Score: 0.43 Kappa: 0.24

Train Epoch:5  Accuracy: 0.68   Average Loss: 0.62  Precision: 0.69   F1 Score: 0.48 Kappa: 0.27

Train Epoch:6  Accuracy: 0.68   Average Loss: 0.61  Precision: 0.70   F1 Score: 0.49 Kappa: 0.29

Train Epoch:7  Accuracy: 0.70   Average Loss: 0.60  Precision: 0.73   F1 Score: 0.52 Kappa: 0.32

Train Epoch:8  Accuracy: 0.70   Average Loss: 0.60  Precision: 0.70   F1 Score: 0.55 Kappa: 0.34

Train Epoch:9  Accuracy: 0.70   Average Loss: 0.59  Precision: 0.71   F1 Score: 0.56 Kappa: 0.35

Train Epoch:10  Accuracy: 0.70   Average Loss: 0.59  Precision: 0.72   F1 Score: 0.54 Kappa: 0.34

Train Epoch:11  Acc

array([[33, 34, 31, ..., 33, 34, 34],
       [30, 32, 33, ..., 32, 34, 31],
       [37, 31, 33, ..., 31, 32, 30],
       ...,
       [29, 30, 30, ..., 31, 32, 32],
       [30, 29, 33, ..., 29, 29, 33],
       [32, 37, 36, ..., 34, 34, 36]], dtype=uint8)